In [1]:
import json
import os
from openai import OpenAI               
import pandas as pd
from tqdm import tqdm
import random                  
import time         
from typing import Any, Dict, List
from dotenv import load_dotenv
from gigachat import GigaChat
import re
load_dotenv()

True

In [ ]:
# Пути
INPUT_FILE  = "data/posts/ajtkulov/selected/selected10k_cleaned.jsonl"
OUTPUT_FILE_ALL_COLUMNS = "data/golden/golden_pairs_all_columns.json"
OUTPUT_FILE = "data/golden/golden_pairs.json"
OUTPUT_FILE_CSV = "data/golden/golden_pairs.csv"

# LM Studio сервер
API_BASE    = "http://localhost:1234/v1"          
API_KEY     = "lm-studio"                         # обычно любой строкой, можно оставить
# MODEL_NAME  = "Qwen/Qwen3-30B-A3B-Instruct-2507"
# MODEL_NAME  = "meta-llama-3.1-8b-instruct"
MODEL_NAME  = "glm-4-32b-0414"



TEST_NUMBER_OF_POSTS = 3

# Параметры генерации
NUM_DESCRIPTIONS = 3
# TEMPERATURE      = 0.75
TEMPERATURE      = 0.8
MAX_TOKENS       = 800
TOP_P            = 0.92
MIN_DESCRIPTION_SYMBOLS = 800
MAX_DESCRIPTION_SYMBOLS = 1700


def random_scores() -> tuple[float, float, float]:
    """
    Возвращает кортеж из трёх случайных чисел в диапазонах:
      - первый:  0.00 – 0.25
      - второй:  0.30 – 0.70
      - третий:  0.75 – 1.00
    """
    low    = round(random.uniform(0.00, 0.25), 2)
    medium = round(random.uniform(0.30, 0.70), 2)
    high   = round(random.uniform(0.8, 1.00), 2)
    
    return (low, medium, high)


BATCH_SIZE  = 10                        # сколько постов обрабатывать за один запуск (для отладки)
SAVE_EVERY  = 20                        # сохранять промежуточный результат каждые N постов

In [38]:
PROMPT_TEMPLATE = """\
Ты — генератор синтетических описаний товаров или УСЛУГ строго по заданной степени релевантности.  
Твоя задача — создать описания, которые **точно соответствуют указанному score**, а не просто "какие-то похожие".

На основе поста из Telegram:
«{post_text}»

Сгенерируй РОВНО {num_desc} описаний товаров. Каждое описание должно иметь **строго заданную** степень релевантности:

{relevance_instructions}

Важные правила строгого соблюдения score:
• score = {low_score} → описание **полностью не связано** с постом. Товар должен быть из другой тематики, без единой общей темы, слова или ассоциации. Никаких намёков на содержание поста!
• score = {mid_score} → описание имеет **очень слабую, косвенную связь** (например, общая сфера, но не по сути поста). Связь должна быть минимальной, почти незаметной. Большинство людей сказали бы: "почти не связано".
• score = {high_score} → описание **идеально подходит** к основной идее поста, решает проблему или напрямую связано с ключевыми словами/примерами из текста.

Чтобы заставить тебя строго следовать score:
1. Сначала подумай шаг за шагом: какая главная тема поста? Какие слова/идеи нельзя использовать в низко-релевантных описаниях?
2. Для score ≤ {mid_score} **запрещено** использовать любые ключевые слова, понятия или близкие ассоциации из поста.
3. Описания должны быть **заметно разными** по тематике и стилю.
4. Если описание кажется слишком близким к более высокому score — понизь релевантность намеренно.

ЖЁСТКИЕ ОГРАНИЧЕНИЯ (обязательны к выполнению):
• Длина каждого описания — от {min_description_symbols} до {max_description_symbols}
  символов включительно. Если длина выходит за пределы диапазона, описание считается НЕВАЛИДНЫМ
  и должно быть переписано.

• Описание должно выглядеть как реалистичное карточное описание товара для маркетплейса или услуги
• Естественный, продающий русский язык
• СТРОГО ЗАПРЕЩЕНО использовать:
  те же товары, предметы или услуги, которые прямо или косвенно упомянуты в исходном посте;
  синонимы, переформулировки или частные варианты этих товаров.
• Для описаний со score > 0.5:
  товар обязан быть тематически близким к теме поста,
  но НЕ МОЖЕТ быть тем же самым товаром или его разновидностью.

Перед финальным ответом проверь каждое описание:
    длина в символах,
    отсутствие совпадений с товарами или услугой из поста,
    соответствие формату маркетплейса.

Если хотя бы одно условие нарушено — исправь описание до соответствия.

Примеры РЕАЛЬНЫХ описаний товаров. Изучи их структуру, стиль, как они написаны. При генерации ориентируйся на эти примеры.

• Термометр комнатный:
  Температура и влажность в помещении непосредственно влияют на наше здоровье, 
  резкие перепады могут привести к сухости в горле, жару, 
  аллергии и просто неприятным ощущениям. Датчик температуры и влажности Xiaomi 
  второго поколения создан специально для чуткого наблюдения за изменением микроклимата, 
  а для еще лучшего контроля датчик можно включить в умный сценарий, создавая различные 
  сочетания с другими устройствами. При помощи Bluetooth-шлюза датчик можно использовать 
  в умных сценариях для совместной работы с другими устройствами, чтобы взять микроклимат 
  в помещении под полный контроль и вручную контролировать температуру и влажность для обеспечения самых комфортных показателей. В приложении Mi Home можно настроить сценарий, 
  при котором по достижении определенной температуры автоматически включается или выключается 
  кондиционер, обогреватель или увлажнитель.

• Зеркало для макияжа, зеркало с подсветкой для макияжа настольное косметическое:
  Этот стильный предмет не только красиво дополнит интерьер вашего ванны или спальни, 
  но и станет незаменимым помощником для создания идеального макияжа. 
  Зеркало с увеличением, предлагает вам возможность видеть каждую деталь вашего образа 
  с удивительной четкостью. Круглое зеркало с подсветкой для макияжа с увеличением, создаст идеальное освещение,
  позволяющее точно наносить макияж и ухаживать за кожей.
  Комплектация: 
  - зеркало 1 шт;
  - мини зеркало 1 шт;
  - провод usb 1 шт.

  
• Себозол противогрибковый шампунь от перхоти, против псориаза, себореи, корочек, лишая с кетоконазолом 100мл:
Шампунь для наружного применения в дерматологии и косметологии. Оказывая противомикробное и противогрибковое действие, устраняет не только симптомы, но и причину появления перхоти.
Шампунь "Себозол" разработан специально для удаления перхоти (воздействует на грибковые 
поражения кожи головы, лица и туловища). 
Его можно использовать при заболеваниях, 
сопровождающихся следующими проявлениями: перхоть, себорейный дерматит 
(красно-коричневые бляшки с шелушением), отрубевидный лишай, себорейный псориаз.
  
• Ортодонтические резинки для брекетов / Тяги для брекетов (эластики) - Медведь / Bear (6,35 мм., 130 гр.) Ormco:

Ортодонтические эластики для брекетов Медведь / Bear из серии Zoo ("Зоопарк")
 произведены одним из самых известных и надежных брендов - Ormco. 
 Резинки Ормко отличаются качеством и долговечностью, что делает их популярным выбором среди ортодонтов
   и пациентов по всему миру. По градации разработанной Ormco, тяги для брекетов Медведь относятся 
   к сильным, их тяговое усилие составляет 4,5 oz / 130 гр, а диаметр кольца равен 1/4'' - 6.35 мм. 
   Важно помнить, заявленное усиление тяги для брекетов достигают при растяжении их в 3 раза. 
   Резиночки для брекетов изготовлены из высококачественного хирургического латекса. 
   Эти резинки на зубы являются неотъемлемой частью ортодонтического лечения и играют 
   важную роль в процессе коррекции исправления прикуса и выправления зубов. 
   Одной из основных функций ортодонтических тяг для брекетов является создание тяги и давления на зубы, 
   направляя их в нужное положение.

Примеры описания УСЛУГ:

  •  Рассчитайте примерные затраты и прибыль на Ozon сразу для схемы FBO и FBS — выберите уже существующий товар или заполните характеристики самостоятельно.



    • Меня зовут Анастасия, представляю следующие услуги:

    -Эпиляция (шугаринг, воск)
    -Электроэпиляция на аппарате Apilus

    -Брови (моделирование, окрашивание, коррекция, ламинирование).

    -Наращивание и ламинирование ресниц

    -Перманентный макияж (брови, губы, веки).

    Принимаю в уютном кабинете в студии, 10 мин. пешком от м. «Сокол».
    В своей работе использую только высококачественные профессиональные косметические средства и одноразовые расходные материалы. Все инструменты проходят 3х-этапнтую стерилизацию.

После генерации проверь, что все условия соблюдены. Если нет, переделай.

Верни ВАЛИДНЫЙ JSON-массив без обрезок. Закрой все скобки/кавычки. Не используй trailing commas. Выводи ТОЛЬКО JSON-массив и ТОЛЬКО после проверки, ничего больше:
[
  {{"description": "...", "relevance_label": "label", "score": 0.XX}},
  ...
]
"""

In [23]:
def get_relevance_instructions(scores: tuple[float, float, float]) -> str:
    """
    Возвращает строку с инструкциями по уровням релевантности.
    
    """
    RELEVANCE_LEVELS = [
    {"label": "совершенно нерелевантно",   "score": scores[0]},
    # {"label": "слабо релевантно",          "score": 0.35},
    {"label": "средне релевантно",         "score": scores[1]},
    {"label": "очень релевантно / идеально", "score": scores[2]},
]
    if len(scores) != len(RELEVANCE_LEVELS):
        raise ValueError(f'Ожидается кортеж ровно из {len(RELEVANCE_LEVELS)} значений score')
    
    defs = [
        f"совершенно нерелевантно (score: {scores[0]:.2f}): Товар из другой тематики, без какой-либо связи с постом (например, если пост про AI — товар про кухню).",
        f"средне релевантно (score: {scores[1]:.2f}): Частичная связь, но не полная. Здесь в части релевантности ориентируйся на то, какой score указан в сскобках (score: 0.XX)!",
        f"очень релевантно (score: {scores[2]:.2f}): Прямая связь с основной идеей поста (например, товар, который решает проблему из поста или связан с примером).",
    ]
    return "\n".join(f"• {defs[i]}" for i in range(len(RELEVANCE_LEVELS)))

In [5]:
def get_few_shot_examples() -> str:
    return """\
- Для нерелевантного: Пост про AI — описание "Кофеварка эспрессо De'Longhi, 15 бар, автоматический капучинатор, цена 12 000 руб." (никакой связи).
- Для средне релевантного: Пост про AI — описание "Курс по Python для начинающих, 50 уроков, цена 2 000 руб." (связь с программированием, косвенно к AI).
- Для очень релевантного: Пост про AI — описание "Подписка на ChatGPT Plus, интеграция нескольких моделей в чат, цена 1 500 руб/мес." (прямая связь с функцией поста).\
"""

In [46]:
# клиент для локальных моделей
client = OpenAI(base_url=API_BASE, api_key=API_KEY)

In [40]:
# загружает отобранные 10к постов
def load_jsonl_to_dicts(input_file: str) -> List[Dict[str, Any]]:
    data = []
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if line:  # пропускаем пустые строки
                    try:
                        item = json.loads(line)
                        data.append(item)
                    except json.JSONDecodeError as e:
                        print(f"Ошибка парсинга JSON в строке {line_num}: {e}")
                        print(f"Проблемная строка: {line[:100]}...")
                        continue
        print(f"✓ Загружено {len(data):,} записей из {input_file}")
        return data
    except FileNotFoundError:
        print(f"✗ Файл не найден: {input_file}")
        return []
    except Exception as e:
        print(f"✗ Ошибка при чтении файла {input_file}: {e}")
        return []
posts = load_jsonl_to_dicts(INPUT_FILE)
# posts = posts[:TEST_NUMBER_OF_POSTS]
posts = random.sample(posts, 3)
print(posts)
print('length:', len(posts))

✓ Загружено 10,029 записей из data/posts/ajtkulov/selected/selected10k_cleaned.jsonl
[{'channel': 'gene_land', 'text': 'Мы работаем над скинченджером (онлайновый - видят все те кто играет с мода)Его текущая версия доступна в VIP версии мода (170 рублей)Также в VIP, можно поменять возраст аккаунта, что даст вам возможность писать в чат, если вы не можете.Информация об покупке випа тут', 'link': 'https://t.me/gene_land/659', 'id': 659, 'date': '2024-03-22 16:06:46+00:00', 'views': '65.6K', 'category': 'Игры', 'original_length': 266, 'cleaned_length': 266}, {'channel': 'StanislavaBAIKAL', 'text': 'Доброе утроЕсли Инстаграмм удалят, мы сможем продолжить с вами общение здесьдавно хотела его завестинастало время, да и потом, здесь можно выкладывать гораздо больше и никто тебя за это не заблокирует', 'link': 'https://t.me/StanislavaBAIKAL/3', 'id': 3, 'date': '2022-02-25 01:06:23+00:00', 'views': '58.3K', 'category': 'Семья и дети', 'original_length': 206, 'cleaned_length': 200}, {'channel': 

In [ ]:
posts = [
# {"channel": "Alisa_Lazarenko", "text": "Девочки ❤️Записала для вас немножко полезностей ☺️✅ Как я рисую стрелки (в стрелках главное тренировки💪🏻 с первого раза мало у кого получается))✅Как правильно прокрашивать реснички✅Как отрастить длинные ресницы😜✅Показала один из любимых тонаков дорогих🙈✅И любимую маску для губСкоро я буду проводить урок макияжа для себя оффлайн 🙌🏻 2 часа полезной информации , секретов и нюансов красивого макияжа на каждый день🥰", "link": "https://t.me/Alisa_Lazarenko/146", "id": 146, "date": "2022-03-31T08:52:47+00:00", "views": "26.7K"},
# {"channel": "copyme", "text": "Прочитала в немецком Elle, что на столе у Миуччи Прады лежит маленькая книжица японского художника Сандзо Вада — «Словарь цветовых комбинаций», составленный еще в 1930-х годах. Это такая энциклопедия из трех сотен удивительных сочетаний оттенков, который Вада использовал для узоров кимоно. Читатели подсказывают, что вот по этой ссылке можно подбирать предложенные художником комбинации онлайн.", "link": "https://t.me/copyme/4122", "id": 4122, "date": "2023-04-05T08:29:12+00:00", "views": "71.2K"},
# {"channel": "cybersrach", "text": "Офлейнера Soniqs кикнули из команды из-за названия статуиВ матче против BOOM на англоязычной трансляции TI обсервер кликнул на статую Leslao, которая называется «Run nigga». После этого игрок попытался извиниться в твиттере:«Только что увидели сообщение на моей статуе. Оно явно неуместно и может задеть, прошу прощения за то, что я вообще написал его. Я небрежно написал это много лет назад, а потом забыл. Глубоко сожалею, что не проверил и не удалил это раньше»Но, извинения не помогли. Сегодня его кикнули из Soniqs 😨", "link": "https://t.me/cybersrach/2481", "id": 2481, "date": "2022-10-19T06:56:37+00:00", "views": "21.3K"},
{"channel": "cybersrach", "text": "Ночью Вышел патч 7.35d. В патче два основных изменения: 🔹Теперь юзеры Dota Plus видят информацию о найденной игре и могут отклонить ее, не получив за это наказание🔹Теперь каждый игрок банит 4 персонажей ДО поиска игры. (1 из которых будет запрещен перед началом)Также затронули метовых героев и предметы: ослабили Black King Bar, Divine Rapier и Revenant’s Brooch. Из героев: Dragon Knight, Terrorblade, Mirana и других", "link": "https://t.me/cybersrach/6815", "id": 6815, "date": "2024-03-22T06:44:07+00:00", "views": "21.0K"},
# {"channel": "cybersrach", "text": "ПАТЧ – ЗАВТРАВ 1 акт «Павшей короны» добавили недостающую свечу, с помощью которой можно увидеть описание нового места и даже часть изменений нового патча:🔹 Базовая броня у ___ уменьшена на 0,5.🔹 Каждый раз, когда выходит солнце, ___ раскрывает всю карту на 5 секунд.🔹 Disruptor: Kinetic Field заменен на Kinetic ___🔹 Снаряды вражеской атаки в радиусе 500 от ___ замедляются.🔹 ___ может путешествовать по дуге🔹 ___ может использовать предметы в своем рюкзаке, как если бы они были в их инвентаре🔹 Silencer не может быть ___🔹 ___ не имеет уровней прокачки на 1 уровне🔹 Lifestealer навсегда получает ___ при убийстве крипа🔹 ___ начинает игру с дополнительными 250 золота🔹 Враги внутри ___ не могут видеть союзниковМнения?", "link": "https://t.me/cybersrach/7278", "id": 7278, "date": "2024-05-22T05:14:07+00:00", "views": "18.7K"},
# {"channel": "cybersrach", "text": "BetBoom распустила состав по Доте 😰«Сегодня мы прощаемся и говорим спасибо составу BetBoom Team. SoNNeikO, Daxak, Larl, Noticed и RodjER покидают нашу команду, но остаются в сердечках всех наших фанатов Желаем парням добиться ещё больших успехов и верим, что они ещё порубятся за титулы на тир-1 сцене   И может быть — против уже другой BetBoom Team   Спасибо, что были с нами!» – написал клуб", "link": "https://t.me/cybersrach/2642", "id": 2642, "date": "2022-11-03T18:44:54+00:00", "views": "17.4K"},
# {"channel": "cybersrach", "text": "Общий сбор! БП вышел 🤩«Откройте для себя новый боевой пропуск, поделённый на две части. Первая из них посвящена чествованию The International, с завершением которого и начнётся вторая. В ней мы отметим завершение турнирного сезона в морозной вариации классического «Восстания Тьмы». Следуя ежегодной традиции, пропуск наполнен первоклассными сокровищницами и функциями — как новыми, так и старыми, — которые точно наполнят вашу игру новыми украшениями и весельем» – Valve", "link": "https://t.me/cybersrach/2106", "id": 2106, "date": "2022-09-01T22:14:43+00:00", "views": "16.3K"},
# {"channel": "nachinay_vospitanie", "text": "Ты лежишь пока в своей кроватке И воркуешь беззаботно-сладко, Ты смеешься маме, погремушкам, Маленький сынок мой, лопотушка. Ты пока лишь с маминых ладоней Видишь мир, глазастик изумленный, Разноцветный, яркий и прекрасный, Каждый день все новый, звонкий, ясный. Но придет тот день и ты на ножки Встанешь неуверенно немножко, И пойдешь из комнаты в другую, А потом за дверь, во двор, ликуя. С каждым шагом все прочнее ноги, С каждым днем заманчивей дороги, С каждым годом все теснее дома, Неизбежно это и знакомо. А пока… и я смотрю на кроху, Нам для счастья нужно так немного, Молоко, игрушки, папа с мамой, Не спеши расти, мой самый-самый!", "link": "https://t.me/nachinay_vospitanie/482", "id": 482, "date": "2023-11-25T10:55:53+00:00", "views": "22.4K"},
# {"channel": "nachinay_vospitanie", "text": "Отложите в сторону дела, И найдите несколько минут, Чтобы детям подарить тепла, А дела, конечно, подождут… Обнимите сына или дочь, Расскажите о своей любви, Время метеором летит прочь, Совсем скоро вырастут они. Не ругайте вы по пустякам Тех, что дарят вам свой яркий свет. Дети же всем сердцем верят вам. Посидите с ними тет-а-тет, Загляните искренне в глаза, И прижмите крепко их к груди, Подберите добрые слова, Расскажите о своей любви! Не стесняйтесь чувства проявлять, Не держитесь вы от них в тени, Детям очень-очень важно знать, Что они на свете не одни. И что рядом с ними есть всегда Твердая опора их семьи, Отложите в сторону дела, И побудьте с вашими детьми!", "link": "https://t.me/nachinay_vospitanie/577", "id": 577, "date": "2024-03-06T12:23:11+00:00", "views": "21.4K"},
# {"channel": "nachinay_vospitanie", "text": "Cыну...  Ты вырастешь, и я не буду знать, С кем ты проводишь дни и даже ночи, Но я все так же буду называть Тебя как в детстве - милый мой сыночек.  Ты вырастешь... Ты будешь принимать Судьбы удары, думаю, достойно. И зубы стискивать и кулаки сжимать, А я уже не поцелую там, где больно.  И если с папой мы не справимся уже. Там кран потек или другие беды, Твой голос в трубке твердо скажет мне: \"Мамуль, не трогай ничего - сейчас приеду\".  Ну, а пока - пижамка в облаках, И на ночь про слона и Айболита. Твое сердечко в маминых руках, Душа еще ранима и открыта.  Я очень постараюсь, мой родной, Дать, что смогу, и отвести ненастья: Чтоб детство ты с улыбкой вспоминал, Чтоб знал, каким бывает счастье...", "link": "https://t.me/nachinay_vospitanie/599", "id": 599, "date": "2024-03-31T07:34:55+00:00", "views": "19.8K"},
# {"channel": "nachinay_vospitanie", "text": "Ты лежишь пока в своей кроватке И воркуешь беззаботно-сладко, Ты смеешься маме, погремушкам, Маленький сынок мой, лопотушка. Ты пока лишь с маминых ладоней Видишь мир, глазастик изумленный, Разноцветный, яркий и прекрасный, Каждый день все новый, звонкий, ясный. Но придет тот день и ты на ножки Встанешь неуверенно немножко, И пойдешь из комнаты в другую, А потом за дверь, во двор, ликуя. С каждым шагом все прочнее ноги, С каждым днем заманчивей дороги, С каждым годом все теснее дома, Неизбежно это и знакомо. А пока… и я смотрю на кроху, Нам для счастья нужно так немного, Молоко, игрушки, папа с мамой, Не спеши расти, мой самый-самый!", "link": "https://t.me/nachinay_vospitanie/527", "id": 527, "date": "2024-01-15T12:16:59+00:00", "views": "17.7K"},
# {"channel": "nachinay_vospitanie", "text": "КАКИЕ ДЕЛА МОЖНО ПОРУЧИТЬ ДЕТЯМ РАЗНОГО ВОЗРАСТА 📌Кapточки-шпаргалки в помощь для родителей📌Многие родители считают, что работа по дому будет отнимать у детей беззаботное детство, которое даётся только раз. У психологов иная точка зрения: когда дети помогают взрослым, они чувствуют себя полноправными членами семьи, учатся быть самостоятельными и не бояться ответственности.", "link": "https://t.me/nachinay_vospitanie/29", "id": 29, "date": "2022-08-25T04:02:44+00:00", "views": "12.8K"},
# {"channel": "Alisa_Lazarenko", "text": "Сегодня читала в инсте историю моей талантливой коллеги визажиста из Харькова … Ей кто то собрал ее кейс и передал, но доехало до неё практически ничего😭😭😭Все самое важное и ценное , кто-то в цепочке передающих спиздил 🤬Как вообще такое возможно? Это днище. Обычный человек не сможет всем этим воспользоваться…нет слов.", "link": "https://t.me/Alisa_Lazarenko/75", "id": 75, "date": "2022-03-16T21:09:05+00:00", "views": "35.0K"},
# {"channel": "Alisa_Lazarenko", "text": "Девочки! Нужна помощь🙏🏻 Набралась храбрости попросить 🙏🏻 помощь заключается в том,что помочь найти, именно найти! (А я куплю) две белые кровати с матрасами односпальные (б/у) в нормальном состоянии. Типа икеа или икеа🙏🏻 сама ищу,но пока безрезультатно (( Может кто то видел  или продаёт лично.🙏🏻 (Москва)", "link": "https://t.me/Alisa_Lazarenko/47", "id": 47, "date": "2022-03-12T16:04:28+00:00", "views": "31.4K"},
# {"channel": "Alisa_Lazarenko", "text": "Кровати приехали так быстро😳💪🏻И это все благодаря ВАМ!!! (Они новые 5000 рублей за кровать) уже собирают🙏🏻сейчас родители привезу детей на смотрины☺️Сегодня одна добрая девушка Юля привезла детский ортопедический стул и много всего для кухни🙏🏻Я думаю, ещё пару дней и можно будет жить 🤞🏻Спасибо вам девочки за вашу поддержку,помощь,денежные переводы🙈🙏🏻❤️", "link": "https://t.me/Alisa_Lazarenko/53", "id": 53, "date": "2022-03-13T14:42:42+00:00", "views": "30.8K"},
# {"channel": "Alisa_Lazarenko", "text": "Два года назад у меня был жуткий стресс, я бы сказала потрясение 🙈Думала конец света, но как видите я жива😆 Так вот, я в тот период ходила по 2-3 тренировки в день каждый день 🤪 И только этим и спасалась… сегодня мама меня вытащила на тренировку💪🏻какой же это кайф 😻 девочки, как часто вы ходите на треши и ходите ли вообще?👀", "link": "https://t.me/Alisa_Lazarenko/105", "id": 105, "date": "2022-03-22T09:19:06+00:00", "views": "30.3K"},
# {"channel": "autohaykcatalog", "text": "Hyundai Elantra Sel 2018❗️В наличии❗️VIN - 5NPD84LF1KH448223✔️ Объем двигателя - 2.0 L✔️ Мощность - 150 л.с.✔️ Бензиновый двигатель✔️ Расход на 100 км. - 7.0 л.✔️ Пробег - 111200 км.✔️ Передний привод✔️ Коробка - автомат (АКПП)✔️ Электроусилитель руля✔️ Дисковые тормоза✔️ Галогенные фары✔️ Датчик света✔️ Датчик дождя✔️ Электропривод боковых зеркал✔️ Электроподогрев зеркал✔️ Автозатемнение зеркал✔️ Обогрев заднего стекла✔️ Мультируль✔️ Бесключевой доступ✔️ Электроподогрев передних сидений✔️ Электрические стеклоподъемники передние✔️ Электрические стеклоподъемники задние✔️ Системаконтроля»слепых зон»✔️ Адаптивные фары✔️ Система контроля полосы✔️ Адаптивный круиз-контроль✔️ Климат-контроль✔️ КондиционерЦена - 17.500$", "link": "https://t.me/autohaykcatalog/7298", "id": 7298, "date": "2022-07-23T11:18:45+00:00", "views": "9.4K"},
# {"channel": "autohaykcatalog", "text": "Kia Optima Lx 2,4 2020 годЦелая Optima 🔥Старт аукциона - через 3 дня#лота - 48710902VIN - 5XXGT4L34LG435948✔️ Объём двигателя - 2,4L✔️ Передний привод✔️ Пробег - 57.500 км✔️ Камера заднего обзора✔️ Мультируль✔️ Автозатемнение зеркалЦена - 20.000$Примерная цена под ключ на учёт России (с учётом ремонта)Успей заказать", "link": "https://t.me/autohaykcatalog/4937", "id": 4937, "date": "2022-06-19T14:22:37+00:00", "views": "9.1K"},
# {"channel": "autohaykcatalog", "text": "https://bid.cars/ru/lot/1-66859271/2016-Mitsubishi-Outlander-JA4AP3AU7GZ0658352016 год17.000$Mitsubishi Outlander Sport Es 2016❗️В наличии❗️Цена в Армении❗️VIN - JA4AP3AU7GZ065835✔️ Объем двигателя - 2.0 L✔️ Мощность - 150 л.с.✔️ Бензиновый двигатель✔️ Расход на 100 км. - 6.3 л.✔️ Пробег - 108100 км.✔️ Передний привод✔️ Коробка - автомат (вариатор)✔️ Система старт-стоп✔️ Электроусилитель руля✔️ Дисковые тормоза✔️ Электрический ручник✔️ Галогенные фары✔️ Электроподогрев зеркал✔️ Автозатемнение зеркал✔️ Обогрев заднего стекла✔️ Электроподогрев лобового стекла✔️ Мультируль✔️ Электроподогрев передних сидений✔️ Электрические стеклоподъемники передние✔️ Электрические стеклоподъемники задние✔️ Круиз-контроль✔️ Раздельный климат-контроль✔️ Камера заднего ходаЦена - 17.000$", "link": "https://t.me/autohaykcatalog/7314", "id": 7314, "date": "2022-07-24T08:35:16+00:00", "views": "8.6K"},
# {"channel": "autohaykcatalog", "text": "время знакомства ⚜🕊Меня зовут Лиза ✔Я сотрудник Ставропольского офиса , компании AUTOHAYK @auto_haykЯ честно и безответно влюблена в свою работу🚘 и стараюсь всегда вложить каждый раз частичку своей души.Обратная связь для меня необходима, все свое внимание стараюсь отдавать работе🚘Я рада каждому из вас , кто находится на нашем аккаунте ⚜Я буду рада ответить на все ваши вопросы , и с радостью просчитаю любой интересующий вас автомобиль📍проспект Кулакова 65📲+7-989-120-00-26", "link": "https://t.me/autohaykcatalog/7374", "id": 7374, "date": "2022-07-25T11:33:14+00:00", "views": "8.5K"},
# {"channel": "autohaykcatalog", "text": "Ford Mustang Ecoboost Premium 2015 ❗️В наличии❗️29.000$ ✔️ Объем двигателя - 2.3 L✔️ Мощность - 305 л.с.✔️ Бензиновый двигатель✔️ Задний привод✔️ Расход на 100 км. - 9.8 л.✔️ Пробег - 190300 км.✔️ Коробка - автомат (АКПП)✔️ Электроусилитель руля✔️ Дисковые тормоза✔️ Ксеноновые фары✔️ Датчик света✔️ Датчик дождя✔️ Электропривод боковых зеркал✔️ Электроподогрев зеркал✔️ Обогрев заднего стекла✔️ Кожаный салон✔️ Мультируль✔️ Электропривод передних сидений✔️ Электроподогрев передних сидений✔️ Вентиляция передних сидений✔️ Бесключевой доступ✔️ Электрические стеклоподъемники передние✔️ Круиз-контроль✔️ Раздельный климат-контроль✔️ Кондиционер✔️ Камера заднего хода", "link": "https://t.me/autohaykcatalog/7680", "id": 7680, "date": "2022-07-31T07:47:54+00:00", "views": "8.3K"},
# {"channel": "arbitraj_delay", "text": "Важно! Рекламодателям!Павел Дуров анонсировал Stories в Телеграм уже в начале июляКак вы понимаете, истории в личных профилях с возможностью репоста из групп увеличат охваты просмотров и актив на страницах и это отличная возможность для вашего бизнеса развивать его в два раза активнееОбращайтесь к нашему менеджеру (в шапке профиля) и он подберёт для вас лучший вариант размещения (у нас больше 3 тысяч каналов с суммарной аудиторией в 150 миллионов человек)P.S. Успейте до глобального повышения цен на фоне увеличения актива =)", "link": "https://t.me/arbitraj_delay/329", "id": 329, "date": "2023-06-27T10:19:33+00:00", "views": "23.0K"},
# {"channel": "arbitraj_delay", "text": "Нейро-художник с помощью Midjourney пофантазировал, как могли бы выглядеть бонусы подписки Плюс, если бы они были буквально связаны. На артах вы можете увидеть Яндекс Музыку, Кинопоиск, баллы Плюса, Букмейт и Плюс Сити.Только посмотрите на эту милейшую колонкуНейро-художник с помощью Midjourney пофантазировал, как могли бы выглядеть бонусы подписки Плюс, если бы они были буквально связаны. На артах вы можете увидеть Яндекс Музыку, Кинопоиск, баллы Плюса, Букмейт и Плюс Сити.", "link": "https://t.me/arbitraj_delay/200", "id": 200, "date": "2023-02-17T16:12:43+00:00", "views": "22.0K"},
# {"channel": "arbitraj_delay", "text": "Парень из США собрал свой ПК из комплектующих, найденных на мусорках возле магазинов техники – на этого к него ушло полгода.Внутри: видеокарта EVGA GTX 570, Intel Core I7-3770 Non-K, 16 ГБ Corsair Vengeance , блок питания на 750-ватт от Corsair, корпус iBUYPOWER Snowblind Element, материнка Dell OptiPlex 9010. Рабочий БП и ОЗУ найти не удалось - он их докупил за 140$.", "link": "https://t.me/arbitraj_delay/255", "id": 255, "date": "2023-04-15T18:53:18+00:00", "views": "21.7K"},
# {"channel": "singularityp0int", "text": "🧬 Чат-бот Илона Маска #Grōk стал доступен для платных подписчиков 💬 в Европе Grok отличается от других ассистентов тем, что предоставляет информацию в реальном времени, используя Twitter/X в качестве базы знаний, стремится быть максимально полезным, остроумным, и не боится острых вопросов или спорных тем. Grōk всё ещё находится в разработке и может не всегда понимать, что вы от него хотите. Например, он соглашается всегда писать на русском, вместо английского, но спустя пару сообщений забывает об этом.🧩 #AINews", "link": "https://t.me/singularityp0int/3860", "id": 3860, "date": "2024-05-19T10:15:04+00:00", "views": "10.5K"},
# {"channel": "singularityp0int", "text": "🧬 Очередной свеженький #КаталогНейросетей: более 4000 актуальных нейросетей в одном месте!gpte.ai — хранит в себе все топовые нейросети из разных сфер: программирование, дизайн, бизнес, текста и т.д. Есть как платные, так и бесплатные, найти что-то под свои нужды вы точно сможете. Также есть удобный поисковик.🧩 #КаталогНейросетей", "link": "https://t.me/singularityp0int/2974", "id": 2974, "date": "2024-02-11T12:00:39+00:00", "views": "10.3K"},
# {"channel": "singularityp0int", "text": "🧬 Олимпиаду в Париже будет комментировать нейросетьПо сообщениям СМИ в роли комментатора на грядущих Играх выступит цифровой ИИ-клон 79-летнего спортивного диктора и телеведущего Эла Майклза.Нейронке собираются скормить около 7 миллионов фраз, которые она будет использовать во время спортивных выступлений. Сам Майклз вначале был настроен к этому скептически, но затем послушал демонстрацию технологии и сказал: «Я в деле!».🧩 #AINews", "link": "https://t.me/singularityp0int/4368", "id": 4368, "date": "2024-07-07T13:20:02+00:00", "views": "10.2K"},
# {"channel": "singularityp0int", "text": "🧬 #OpenAI официально запустили функцию вызова разных GPT в одном чате.Набрав \"@\", а затем название GPT, можно вызвать в один чат разные GPT. Это позволяет создать более персонализированного универсального ассистента в одном окне, который сможет связывать ответы и навыки каждого GPT друг с другом. На примере представлена словесная дуэль чат-ботов с личностями Трампа и Байдена. По тексту видно, что GPT ссылаются на слова друг друга.🧩 #AINews", "link": "https://t.me/singularityp0int/2886", "id": 2886, "date": "2024-02-02T09:44:01+00:00", "views": "10.0K"},
# {"channel": "singularityp0int", "text": "🧬 Google Cloud и Hugging Face стали стратегическими партнерами.Популярная платформа для машинного обучения получит доступ к вычислительным ресурсам Google Cloud и платформе для тренировки и развертывания моделей Vertex AI. Сообщается, что сотрудничество с Google будет в области с открытым научным исследованием, с открытым исходным кодом, облачных технологий и оборудования, чтобы компании могли создавать свои собственные ИИ, используя новейшие открытые модели от Hugging Face и новейшие облачные и аппаратные функций от Google Cloud.Обе компании обещают, что новое партнерство позволит сделать ИИ с открытым исходным кодом еще доступнее.🧩 #AINews", "link": "https://t.me/singularityp0int/2868", "id": 2868, "date": "2024-01-31T07:37:02+00:00", "views": "9.9K"},
# {"channel": "emperia_film", "text": "УЖЕ СЛЫШАЛИ ОБ ЭТОМ❓Оригинальная техника Apple прямиком из Европы!  - ПО ЦЕНЕ НИЖЕ РЫНКА ❗️ТАКОГО ВЫ ТОЧНО НЕ НАЙДЁТЕ НИ В ОДНОМ МАГАЗИНЕ И МАРКЕТПЛЕЙСЕ! -Айфоны 📱 -Макбуки 👩‍💻 -Часы ⏰ -Планшеты 📱 Эти ребята отвечают за качество и дают заводскую гарантию!*а ещё можно купить оптом🤫Скорее переходи в канал, пока он открыт 💪https://t.me/Smarton_deviceA1032", "link": "https://t.me/emperia_film/4667", "id": 4667, "date": "2024-02-26T17:00:10+00:00", "views": "7.4K"},
# {"channel": "emperia_film", "text": "Фильм №22 - Келин особого назначения (2022)#комедия Сотрудница спецслужб Асель по ошибке принимает за бандита обычного парня Ерлана, который увозит её в деревню в качестве будущей невесты. Теперь, чтобы вывести подозреваемого и его семью на чистую воду, девушка должна вести домашнее хозяйство, о котором совсем ничего не знает.🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/244", "id": 244, "date": "2023-02-22T05:30:09+00:00", "views": "7.0K"},
# {"channel": "emperia_film", "text": "Фильм №186 -  Один вдох (2020)#драма В сорок лет Марина Гордеева неожиданно открывает для себя фридайвинг – экстремальное подводное плавание. Погружаясь на глубину, она бросает вызов своему страху и пределу человеческих возможностей. Виктория Исакова, Владимир Яглыч, Максим Суханов, Артем Ткаченко и Стася Милославская в перехватывающей дыхание спортивной драме Елены Хазановой «Один вдох». Прототипом главной героини стала «королева фридайвинга» Наталья...🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/2319", "id": 2319, "date": "2023-07-09T20:59:34+00:00", "views": "6.8K"},
# {"channel": "emperia_film", "text": "Фильм №17 - Убийство онлайн(2021)#триллеры Действие остросюжетного триллера разворачивается в 2022 году. Пандемия COVID-19 всё ещё бушует в мире, и в США находится в затяжном карантине. На этом фоне семеро друзей, живущие в разных уголках страны, решили всё-таки провести вечеринку, но в режиме онлайн, чтобы поздравить одного из героев Эвана с днём рождения.🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/201", "id": 201, "date": "2023-02-19T18:31:42+00:00", "views": "6.6K"},
# {"channel": "emperia_film", "text": "Фильм №40 -  На склоне (2020)#драма Благополучная семейная пара Пит и Билли приезжают вместе с детьми на горнолыжный курорт. Неожиданно они оказываются в экстремальной ситуации, пережив сход лавины, которая погребает под собой их счастливый брак. Оказывается, при наступлении опасности самые близкие люди могут вести себя совсем не так, как мы от них ожидаем…🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/367", "id": 367, "date": "2023-02-28T07:31:09+00:00", "views": "6.5K"},
# {"channel": "copyme", "text": "Time опубликовали первый отрывок из биографии Анны Винтур. Он как раз рассказывает об организации Met Gala. Есть смешной момент, когда Анна просила помощников наконец-то усадить Ким Кардашьян за стол, но хозяйка бала не учла, что в этом платье Mugler Кардашьян просто физически не могла сесть. Зачем-то припомнили Харви Вайнштейна. Говорят, он лихорадочно добивался расположения Анны еще с середины 1990-х и в итоге добился: по крайней мере на Met Gala Анна распоряжалась, чтобы его пропускали сразу, вытаскивая из очереди на красную дорожку.", "link": "https://t.me/copyme/1569", "id": 1569, "date": "2022-04-26T15:11:23+00:00", "views": "109.3K"},
# {"channel": "copyme", "text": "Умеет же Джонатан Андерсон выбирать героев :) Это женщина в новой рекламе Loewe — художница по костюмам Сэнди Пауэлл. Она отвечала за образы в «Орландо», «Волке с Уолл-Стрит», «Фаворитке» — всего у Сэнди 15 номинаций на «Оскар» и три победы. Пару лет назад она посещала киноцеремонии в светлом костюме из простого сукна и с черными фломастерами в карманах — и просила знаменитостей расписаться на пиджаке и брюках. В итоге костюм вместил более 200 подписей, включая Ди Каприо, Скарлетт Йоханссон, Роберта де Ниро, Скорсезе и Тильду. Дальше Пауэлл отправила костюм на аукцион — а на вырученные деньги спасла резиденцию своего друга, режиссера Дерека Джармена, которую хотели снести. Сейчас костюм Сэнди Пауэлл хранится в Музее Виктория и Альберта.", "link": "https://t.me/copyme/2635", "id": 2635, "date": "2022-11-14T11:23:42+00:00", "views": "76.8K"},
# {"channel": "copyme", "text": "«Я обычная девчонка из Техаса, и на мне платье Schiaparelli, которое сделал мой брат Дэниел!». В тиктоке взлетело трогательное видео со свадьбы некой Лиз Фокс Розберри — как оказалось, сестры креативного директора Schiaparelli Дэниела Розберри. На бирке платья дизайнер собственноручно вышил слово Зибо — детское прозвище сестренки.", "link": "https://t.me/copyme/1637", "id": 1637, "date": "2022-05-12T21:05:28+00:00", "views": "66.2K"},
# {"channel": "copyme", "text": "Здесь даже без шуток — дизайнер Сабато де Сарно придумал эти туфли Gucci, когда недавно в Будапеште встретил двух венгерских овчарок (комондор). Сабато в жизни таких не видел — в его родном Неаполе, говорит, таких нет, у них там либо низкорослые борзые в ошейниках со стразами, либо свирепые мастифы, которые охраняют виллы с видом на Везувий. «Первая мысль была — сделать туфли в честь этих венгерских собак. Смешные свисающие туфли».", "link": "https://t.me/copyme/7791", "id": 7791, "date": "2024-03-25T11:21:18+00:00", "views": "54.8K"}
]

print(f"Загружено постов: {len(posts):,}")
print("Пример первого поста:", posts[0]["text"][:120], "...")

### Генерация описаний товаров для поста (с использованием **локальной LLM**)

Эта функция — ключевая часть пайплайна по созданию синтетических пар «пост — товар» с метками релевантности.  
Она берёт текст одного Telegram-поста и запрашивает **локальную LLM** (развёрнутую на твоей машине, например через Ollama, LM Studio или llama.cpp) сгенерировать сразу несколько описаний товаров с разной степенью релевантности.

**Что именно делает функция:**

1. Формирует сложный, хорошо структурированный промпт с помощью шаблона `PROMPT_TEMPLATE`:
   - вставляет текст поста
   - указывает, сколько описаний нужно сгенерировать (`NUM_DESCRIPTIONS`)
   - передаёт инструкции по уровням релевантности (low/mid/high score)
   - добавляет few-shot примеры (через `get_few_shot_examples()`)
   - задаёт ограничения по длине описания (`MIN_DESCRIPTION_SYMBOLS` — `MAX_DESCRIPTION_SYMBOLS`)

2. Отправляет промпт в **локальную модель** через клиент:
   - модель: `MODEL_NAME`
   - параметры: температура, max_tokens, top_p — для контроля креативности и длины

3. Обрабатывает ответ:
   - убирает возможные markdown-обёртки (```json … ```)
   - парсит JSON в список словарей
   - проверяет, что вернулось ровно нужное количество описаний

4. Каждый элемент возвращаемого списка — это словарь примерно такого вида:
   ```python
   {
       "description": "текст описания товара",
       "relevance_label": "low/mid/high" или строка с объяснением,
       "score": 0.1 / 0.5 / 0.9 (число от 0 до 1)
   }

In [ ]:
def generate_product_descriptions(post_text: str):# -> list[Any]:# -> list[Any]: # type: ignore
    """Возвращает список словарей с description, relevance_label, score"""
    
    SCORES = random_scores()

    prompt = PROMPT_TEMPLATE.format(
        post_text = post_text.strip(),
        num_desc  = NUM_DESCRIPTIONS,
        relevance_instructions = get_relevance_instructions(SCORES),
        few_shot_examples = get_few_shot_examples(),
        min_description_symbols = MIN_DESCRIPTION_SYMBOLS,
        max_description_symbols = MAX_DESCRIPTION_SYMBOLS,
        low_score=SCORES[0],
        mid_score=SCORES[1],
        high_score=SCORES[2]
    )

    print(prompt)
    
    try:
        resp = client.chat.completions.create(
            model       = MODEL_NAME,
            messages    = [{"role": "user", "content": prompt}],
            temperature = TEMPERATURE,
            max_tokens  = MAX_TOKENS,
            top_p       = TOP_P,
        )
        
        text = resp.choices[0].message.content.strip()
        
        # Убираем возможные ```json ... ``` обёртки
        if text.startswith("```json"):
            text = text.split("```json", 1)[1].split("```", 1)[0].strip()
        elif text.startswith("```"):
            text = text.split("```", 2)[1].strip()
        
        data = json.loads(text)
        
        if not isinstance(data, list) or len(data) != NUM_DESCRIPTIONS:
            print("Неверный формат ответа →", text[:200])
            return []
            
        return data
        
    except Exception as e:
        print("Ошибка при генерации:", str(e))
        return []

### Основной цикл генерации синтетических пар «пост — описание товара» (с локальной LLM)

Эта ячейка — **сердце этапа синтетической разметки**.  
Она берёт список постов (из ранее отобранного датасета) и для **каждого поста** вызывает локальную LLM, чтобы сгенерировать несколько описаний товаров с разной степенью релевантности. Затем формирует из них готовые пары для последующего обучения моделей (cross-encoder → bi-encoder).

**Что именно происходит в ячейке:**

1. **Инициализация пустого списка** `output_pairs` — сюда будут собираться все сгенерированные пары.

2. **Цикл по всем постам** с прогресс-баром `tqdm`:
   - Для каждого поста берётся текст (`post["text"]`)
   - Вызывается функция `generate_product_descriptions()` → возвращает список из `NUM_DESCRIPTIONS` описаний товаров  
     (каждое с полями: `description`, `relevance_label`, `score`)

3. **Проверка качества генерации**:
   - Если модель вернула не ровно `NUM_DESCRIPTIONS` описаний → выводится предупреждение  
     (это помогает отловить случаи, когда LLM "забыла" формат или обрезала ответ)

4. **Формирование пары для каждой сгенерированной записи**:
   - Создаётся словарь `pair` со следующими полями:
     - `description` — текст описания товара (от LLM)
     - `post_text` — оригинальный текст поста
     - `score` — числовая оценка релевантности (0.0–1.0)
     - `relevance_label` — метка уровня (например, "low", "mid", "high" или текстовое объяснение)
     - Метаданные поста (для трассировки и анализа):
       - `channel` — имя канала
       - `post_id` — ID поста
       - `link` — ссылка на пост/канал
       - `date` — дата публикации
       - `views` — количество просмотров

5. **Сбор всех пар** в общий список `output_pairs`

6. **Промежуточное сохранение** (закомментировано, но готово к использованию):
   - Каждые `SAVE_EVERY` постов можно сохранять промежуточный результат в JSON  
     (очень полезно при больших объёмах — чтобы не потерять прогресс при сбое или перезапуске)

**Зачем это нужно в проекте**

- Из 700k+ постов ты отбираешь ~10k качественных → для них генерируешь синтетические товары  
- Получаешь тысячи пар с **реалистичными** score-метками (gold/silver)  
- Эти пары идеально подходят для:
  - обучения cross-encoder (на 10–30k парах)
  - последующего масштабирования на весь корпус (silver-метки от cross-encoder)
  - обучения bi-encoder в режиме contrastive learning

**Ключевые преимущества подхода:**
- Всё локально → приватность и нулевые затраты на API
- Контролируемое количество описаний на пост (`NUM_DESCRIPTIONS`)
- Сохраняются все метаданные поста → удобно для анализа, фильтрации, отладки
- Устойчиво к ошибкам генерации (пустой ответ → просто пропускаем)

**После выполнения ячейки**  
В переменной `output_pairs` окажется список словарей, готовый к сохранению в JSON/CSV и дальнейшему использованию как тренировочный датасет для моделей релевантности.

In [ ]:
output_pairs: List[Dict[str, Any]] = []

for i, post in enumerate(tqdm(posts, desc="Генерация описаний")):
    descs = generate_product_descriptions(post["text"])

    print(descs)
    
    if len(descs) != NUM_DESCRIPTIONS:
        print(f"Пост {post.get('id', '???')} → получено {len(descs)} описаний вместо {NUM_DESCRIPTIONS}")
    
    for d in descs:
        pair = {
            "description":      d.get("description", "").strip(),
            "post_text":        post["text"],
            "score":            float(d.get("score", 0.0)),
            "relevance_label":  d.get("relevance_label", ""),
            
            # Оригинальные поля поста
            "channel":          post.get("channel", ""),
            "post_id":          post.get("id", None),
            "link":             post.get("link", ""),
            "date":             post.get("date", ""),
            "views":            post.get("views", ""),
        }
        output_pairs.append(pair)


Генерация описаний:   0%|          | 0/20 [00:00<?, ?it/s]

Ты — генератор синтетических описаний товаров строго по заданной степени релевантности.  
Твоя задача — создать описания, которые **точно соответствуют указанному score**, а не просто "какие-то похожие".

На основе поста из Telegram:
«Ссылки на мини-капсулу для читательницы: 1. Черные брюки2. Синий джемпер3. Юбка из кожи и еще одна4. Джинсы5. Белая рубашка6. Пуховик и еще один 7. Шерстяное пальто и еще одно 8. Водолазки синяя и черная9. Лонгслив10. Пиджак11. Кардиган и еще один 12. Шапки синяя и серая13. Шарф и перчатки 14. Ботильоны и ботинки 15. Сумки: синяя (еще синяя), белая, черная 16. Серьги»

Сгенерируй РОВНО 3 описаний товаров. Каждое описание должно иметь **строго заданную** степень релевантности:

• совершенно нерелевантно (score: 0.22): Товар из другой тематики, без какой-либо связи с постом (например, если пост про AI — товар про кухню).
• средне релевантно (score: 0.56): Частичная связь, но не полная. Здесь в части релевантности ориентируйся на то, какой score указан в сск

Генерация описаний:   0%|          | 0/20 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [25]:
_GIGA_CLIENT: GigaChat | None = None
_GIGA_TOKEN_EXPIRES_AT: float = 0.0
_GIGA_CREDENTIALS  =os.getenv("GIGA_CHAT_API_KEY")


### Подготовка клиента GigaChat (для генерации описаний товаров)

Эта ячейка содержит **вспомогательную функцию** `_get_giga_client()`, которая отвечает за создание и кэширование клиента GigaChat (модель от Сбера).

**Зачем это нужно:**

Дальше в ноутбуке идут ячейки, в которых для генерации описаний товаров вместо локальной LLM используется **GigaChat** (через официальный SDK).  
Чтобы не создавать новый клиент и не запрашивать токен при каждом обращении (это дорого по времени и лимитам), здесь реализован **паттерн singleton с кэшированием токена**:

- Клиент создаётся только один раз или когда токен истёк  
- Токен хранится 30 минут (`_GIGA_TOKEN_EXPIRES_AT`)  
- При повторных вызовах возвращается уже готовый клиент с действующим access_token

Это позволяет быстро и стабильно генерировать синтетические данные без лишних авторизаций и сэкономить на лимитах API.

In [26]:
def _get_giga_client() -> GigaChat:
    global _GIGA_CLIENT, _GIGA_TOKEN_EXPIRES_AT

    now = time.time()

    if _GIGA_CLIENT is None or now >= _GIGA_TOKEN_EXPIRES_AT:
        giga = GigaChat(
            credentials=_GIGA_CREDENTIALS,
            verify_ssl_certs=False,
        )
        response = giga.get_token()

        _GIGA_CLIENT = GigaChat(
            access_token=response.access_token,
            verify_ssl_certs=False,
        )
        _GIGA_TOKEN_EXPIRES_AT = now + 30 * 60  # 30 минут

    return _GIGA_CLIENT


### Вспомогательная функция: надёжный парсинг и «починка» ответа от GigaChat

Эта пара функций (`parse_gigachat_response` + `_validate_list`) — важный защитный слой при работе с GigaChat.  
Модель от Сбера часто возвращает ответы в «почти JSON», но с типичными проблемами:  
- markdown-обёртки (```json … ```)  
- лишние префиксы («Вот JSON:», «Ответ:», «Вот массив:» и т.п.)  
- незакрытые массивы (нет финальной `]`)  
- лишние пробелы, пустые строки или обрезанные токены  

Без такой обработки парсинг падает на 15–40% запросов, что приводит к потере данных и остановке генерации.

**Что делает основная функция `parse_gigachat_response(raw_response)`:**

1. **Очистка markdown-обёрток**  
   Убирает ```json … ``` или просто ``` … ```

2. **Удаление типичных префиксов GigaChat**  
   Список `common_prefixes` покрывает самые частые фразы, которые модель любит добавлять перед JSON.

3. **Удаление пустых строк**  
   Оставляет только содержательные линии → чище текст для парсинга.

4. **Простой, но эффективный фикс незакрытого массива**  
   - Если строка начинается с `[` и не заканчивается на `]` → автоматически дописывает `]`  
   - Убирает лишние пробелы после последнего объекта

5. **Парсинг через `json.loads()`**

6. **Валидация результата**  
   Вызывает внутреннюю функцию `_validate_list`, которая проверяет:  
   - вернулся именно список  
   - каждый элемент — словарь  
   - в каждом словаре обязательно есть ключ `"description"`  
   Если хотя бы один элемент не соответствует → выбрасывается ошибка (чтобы сразу заметить проблему)

7. **Обработка всех ошибок**  
   При любом сбое (невалидный JSON, обрезанный текст, неверный формат и т.д.):  
   - выводит сырой ответ (первые 600 символов)  
   - показывает очищенный текст  
   - печатает текст ошибки  
   - возвращает пустой список → цикл генерации не падает, просто пропускает проблемный пост

**Результат успешного выполнения**  
Чистый список словарей, готовый к использованию:

```python
[
    {"description": "Лёгкие беговые кроссовки...", "score": 0.92, "relevance_label": "high", ...},
    {"description": "Кухонный комбайн Bosch...", "score": 0.12, "relevance_label": "low", ...},
    ...
]

In [ ]:
from json_repair import repair_json

def parse_gigachat_response(raw_response: str) -> List[Dict[str, Any]]:
    """
    Улучшенная очистка и парсинг ответа от GigaChat.
    Фиксит обрезки, trailing commas, missing delimiters.
    """
    try:
        text = raw_response.strip()
        
        # 1. Убираем markdown и префиксы (как было)
        if text.startswith("```json"):
            text = text[7:].lstrip()
        elif text.startswith("```"):
            text = text[3:].lstrip()
        if text.endswith("```"):
            text = text[:-3].rstrip()
        
        common_prefixes = [
            "Вот JSON:", "Вот ответ:", "Ответ:", "JSON:",
            "Вот массив:", "Результат:", "Вот:",
        ]
        for prefix in common_prefixes:
            if text.lower().startswith(prefix.lower()):
                text = text[len(prefix):].lstrip()
        
        # 2. Убираем пустые строки
        lines = [line for line in text.splitlines() if line.strip()]
        text = "\n".join(lines).strip()
        
        # 3. 🔥 УЛУЧШЕННЫЙ ФИКС ДЛЯ ОБРЕЗОК И ЗАПЯТЫХ
        # a. Удаляем trailing commas (e.g., "score": 0.20, } -> "score": 0.20 }
        text = re.sub(r',\s*([}\]])', r'\1', text)  # Убираем , перед } или ]
        
        # b. Если обрезано посреди строки (e.g., "использова -> "использова"), закрываем кавычку
        # Ищем незакрытые " и закрываем их, если после них нет пары
        open_quotes = text.count('"') % 2  # Если нечётное кол-во ", закрываем последнюю
        if open_quotes == 1:
            # Находим позицию последней открытой " и добавляем "
            last_open = text.rfind('"')
            if last_open != -1:
                text = text[:last_open + 1] + '"' + text[last_open + 1:]
        
        # c. Если обрезано посреди объекта, обрезаем до последней полной }
        if '{' in text and not text.endswith('}'):
            last_brace = text.rfind('}')
            if last_brace != -1:
                text = text[:last_brace + 1]
                # Добавляем ] если это массив
                if text.startswith('[') and not text.endswith(']'):
                    text += ']'
            else:
                # Если нет ни одной }, это мусор — добавляем базовый []
                text = '[]'
        
        # d. Нормализуем whitespace внутри (убираем лишние \n в ключах/значениях)
        text = re.sub(r'\s+', ' ', text)  # Сжимаем множественные пробелы
        text = re.sub(r'\n\s*', ' ', text)  # Заменяем \n на пробел
        
        # 4. 🔥 СУПЕР-ФИКС: используем json-repair если доступен
        
        text = repair_json(text)
        
        # 5. Финальный парсинг
        parsed = json.loads(text)
        
        # 6. 🔥 ИСПРАВЛЕНИЕ: если parsed[0] - список, берем его (как было)
        if isinstance(parsed, list) and len(parsed) > 0 and isinstance(parsed[0], list):
            parsed = parsed[0]
        
        if isinstance(parsed, list):
            return _validate_list(parsed)  # Твоя функция валидации
        return []
    
    except json.JSONDecodeError as e:
        print("❌ JSONDecodeError в parse_gigachat_response")
        print("Сырой текст (первые 800 символов):", repr(raw_response[:800]))
        print("После всех фиксов:", repr(text[:800]))  # Больше для отладки
        print("Ошибка:", str(e))
        print("Подсказка: Проверь обрезку в конце первого объекта (возможно, нужна ручная правка промпта LLM).")
        return []
    except Exception as e:
        print("❌ Неожиданная ошибка:", str(e))
        return []


def _validate_list(items: list) -> list:
    for item in items:
        if not isinstance(item, dict) or "description" not in item:
            raise ValueError("Элемент не dict или нет ключа 'description'")
    return items

In [28]:
giga = _get_giga_client()

### Асинхронная генерация описаний товаров через GigaChat (с ограничением параллелизма)

Эта функция — **асинхронная версия** генератора описаний товаров, адаптированная специально под GigaChat (API от Сбера).  
Она используется в следующем шаге для параллельной (но контролируемой) обработки большого количества постов.

**Ключевые особенности и зачем это сделано именно так:**

1. **Асинхронность** (`async def`, `await giga.achat`)  
   Позволяет одновременно отправлять запросы к GigaChat без блокировки основного потока → ускоряет обработку тысяч постов в 3–10 раз по сравнению с синхронным циклом.

2. **Глобальный семафор** (`_semaphore = asyncio.Semaphore(1)`)  
   Ограничивает **одновременные запросы к API до 1** (можно увеличить до 2–3, если лимиты аккаунта позволяют).  
   Это критически важно, потому что:  
   - GigaChat имеет строгие rate-limits (обычно 1–5 запросов в секунду в зависимости от тарифа)  
   - без семафора можно легко получить 429 Too Many Requests и бан на время  
   - семафор гарантирует, что запросы идут последовательно или с минимальным параллелизмом

3. **Формирование промпта** — тот же мощный шаблон, что и для локальной LLM:
   - вставляется текст поста  
   - указывается количество описаний (`NUM_DESCRIPTIONS`)  
   - передаются инструкции по уровням релевантности  
   - добавляются few-shot примеры  
   - задаются ограничения по длине описания

4. **Асинхронный вызов** `await giga.achat(prompt)`  
   Здесь используется асинхронный метод клиента GigaChat (предполагается, что `giga` — это экземпляр, полученный из `_get_giga_client()`).

5. **Обработка ответа**:
   - Выводится сырой ответ для отладки (`max raw response:`)
   - Вызывается `parse_gigachat_response()` — функция-парсер с починкой (из предыдущей ячейки)
   - Проверяется, что вернулся именно список
   - При любой ошибке возвращается пустой список → цикл не падает

6. **Защита от сбоев**  
   Весь блок обёрнут в `try-except` → даже если GigaChat вернёт мусор, таймаут или 500-ю ошибку, функция просто вернёт `[]` и обработка продолжится.

**Зачем именно асинхронная версия здесь**

- Предыдущая функция (`generate_product_descriptions`) была синхронной и работала с локальной LLM  
- GigaChat — облачный API → запросы идут по сети, и ждать каждый по очереди слишком долго (особенно на 10k+ постах)  
- Асинхронность + семафор = оптимальный баланс между скоростью и соблюдением лимитов API  
- Позволяет дальше написать эффективный параллельный цикл с `asyncio.gather()` или `tqdm_asyncio`

**Результат выполнения**  
Список из `NUM_DESCRIPTIONS` словарей с описаниями товаров, score и метками релевантности — точно такого же формата, как от локальной модели.  
Готов к сбору в пары «пост — товар — score».

**Примечание**  
Семафор установлен на 1 — это безопасный старт.  
Если ваш тариф GigaChat позволяет 3–5 rps → можно изменить на `Semaphore(3)` и ускорить генерацию в несколько раз.

In [33]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

# Глобальный семафор (создаётся один раз вне функции)
_semaphore = asyncio.Semaphore(1)


async def generate_product_descriptions_gigachat(post_text: str) -> List[Dict[str, Any]]:
        
    async with _semaphore:  # ← здесь семафор, ограничивает параллелизм
        try:
            SCORES = random_scores()
            prompt = PROMPT_TEMPLATE.format(
                post_text=post_text.strip(),
                num_desc=NUM_DESCRIPTIONS,
                relevance_instructions=get_relevance_instructions(SCORES),
                few_shot_examples=get_few_shot_examples(),
                min_description_symbols=MIN_DESCRIPTION_SYMBOLS,
                max_description_symbols=MAX_DESCRIPTION_SYMBOLS,
                low_score=SCORES[0],
                mid_score=SCORES[1],
                high_score=SCORES[2]
            )
            
            # Асинхронный вызов chat
            response = await giga.achat(prompt)
            # await asyncio.sleep(random.uniform(0.8, 10))
            
            # print('max raw response:', response.choices[0].message.content)

            response_descriptions = parse_gigachat_response(response.choices[0].message.content)
            # print('desc:', response_descriptions)
            
            if not isinstance(response_descriptions, list):
                raise ValueError("Ответ не список")
            return response_descriptions
        except Exception as e:
            print("❌ Ошибка парсинга ответа GigaChat")
            print('ошибка:', e)
            return []

### Асинхронный цикл генерации пар «пост — товар» с использованием GigaChat

Эта ячейка — **основной рабочий цикл** для массовой генерации синтетических пар с помощью **GigaChat** (облачная модель от Сбера).  
Она обрабатывает список постов (`posts`) и для каждого поста асинхронно запрашивает описания товаров, собирая результаты в список `output_pairs`.

**Что именно делает ячейка:**

1. **Инициализирует пустой список** `output_pairs` — сюда будут попадать все готовые пары.

2. **Асинхронный цикл по постам** с прогресс-баром `tqdm`:
   - Для каждого поста берётся текст (`post["text"]`)
   - Вызывается **асинхронная** функция `await generate_product_descriptions_gigachat(post["text"])`  
     → возвращает список описаний товаров с метками релевантности и score

3. **Формирование пары для каждого сгенерированного описания**:
   - Создаётся словарь `pair` с ключевыми полями:
     - `description` — текст описания товара от GigaChat
     - `post_text` — полный текст оригинального поста
     - `score` — числовая оценка релевантности (приводится к float)
     - `relevance_label` — текстовая метка уровня релевантности
     - Метаданные поста (для трассировки и анализа):
       - `channel` — имя канала
       - `post_id` — идентификатор поста
       - `link` — ссылка на пост/канал
       - `date` — дата публикации
       - `views` — количество просмотров

4. **Сбор всех пар** в общий список `output_pairs`

**Важные особенности реализации:**

- Используется `await` → функция `generate_product_descriptions_gigachat` асинхронная и защищена семафором  
  → запросы к GigaChat идут **контролируемо** (не больше одного одновременно по умолчанию), что соблюдает rate-limits API и предотвращает ошибки 429 (Too Many Requests)

- Если GigaChat вернул пустой список или произошла ошибка → пост просто пропускается (благодаря обработке внутри функции), цикл продолжается


**Результат выполнения ячейки**  
В переменной `output_pairs` окажется список словарей — готовый датасет синтетических пар:

```python
[
    {
        "description": "Лёгкие беспроводные наушники с шумоподавлением...",
        "post_text": "Ищу хорошие наушники для бега...",
        "score": 0.88,
        "relevance_label": "high",
        "channel": "fitness_channel",
        "post_id": 12345,
        "link": "t.me/fitness_channel/12345",
        "date": "2025-10-15",
        "views": 4500
    },
    ...
]

In [41]:
# posts = posts[:TEST_NUMBER_OF_POSTS]
output_pairs: List[Dict[str, Any]] = []
for post in posts:
    print(post)
for i, post in enumerate(tqdm(posts, desc="Генерация описаний")):
    descs = await generate_product_descriptions_gigachat(post["text"])
    for d in descs:
        pair = {
            "description":      d.get("description", "").strip(),
            "post_text":        post["text"],
            "score":            float(d.get("score", 0.0)),
            "relevance_label":  d.get("relevance_label", ""),
            "channel":          post.get("channel", ""),
            "post_id":          post.get("id", None),
            "link":             post.get("link", ""),
            "date":             post.get("date", ""),
            "views":            post.get("views", ""),
            "category":         post.get("category", "")
        }
        output_pairs.append(pair)

# -----------------------------------------------

pd.set_option("display.max_colwidth", None)
# Быстрый обзор
df = pd.DataFrame(output_pairs)
print("\nРаспределение score:")
print(df["score"].value_counts().sort_index())

print("\nПример 3 случайных записей:")
df[["description", "post_text", "score", "relevance_label"]]
# df

{'channel': 'gene_land', 'text': 'Мы работаем над скинченджером (онлайновый - видят все те кто играет с мода)Его текущая версия доступна в VIP версии мода (170 рублей)Также в VIP, можно поменять возраст аккаунта, что даст вам возможность писать в чат, если вы не можете.Информация об покупке випа тут', 'link': 'https://t.me/gene_land/659', 'id': 659, 'date': '2024-03-22 16:06:46+00:00', 'views': '65.6K', 'category': 'Игры', 'original_length': 266, 'cleaned_length': 266}
{'channel': 'StanislavaBAIKAL', 'text': 'Доброе утроЕсли Инстаграмм удалят, мы сможем продолжить с вами общение здесьдавно хотела его завестинастало время, да и потом, здесь можно выкладывать гораздо больше и никто тебя за это не заблокирует', 'link': 'https://t.me/StanislavaBAIKAL/3', 'id': 3, 'date': '2022-02-25 01:06:23+00:00', 'views': '58.3K', 'category': 'Семья и дети', 'original_length': 206, 'cleaned_length': 200}
{'channel': 'ekspertmp', 'text': 'Выкладываем запись вчерашнего прямого эфираСоветуем просмотреть ег

Генерация описаний: 100%|██████████| 3/3 [00:07<00:00,  2.42s/it]


Распределение score:
score
0.04    1
0.15    1
0.23    1
0.40    1
0.55    1
0.63    1
0.95    1
0.98    2
Name: count, dtype: int64

Пример 3 случайных записей:


,description,post_text,score,relevance_label
0,"Хотите попробовать себя в роли профессионального садовода, даже если у вас нет дачи или огорода? Миниатюрная ферма GrowBox позволит вырастить свежие овощи и зелень прямо дома! Благодаря продуманному освещению, воздухообмену и автоматической подаче удобрений, растение получает всё необходимое для роста и развития. Простота установки и ухода делают эту ферму идеальным подарком для любого человека, стремящегося получать витамины круглый год.","Мы работаем над скинченджером (онлайновый - видят все те кто играет с мода)Его текущая версия доступна в VIP версии мода (170 рублей)Также в VIP, можно поменять возраст аккаунта, что даст вам возможность писать в чат, если вы не можете.Информация об покупке випа тут",0.23,совершенно нерелевантно
1,"Забота о красоте требует особого подхода. Умная расческа Dermaroll поможет сохранить здоровье волос и сделать прическу безупречной. Специальные ролики аккуратно массируют кожу головы, улучшая кровообращение и стимулируя рост волос. Регулярное использование расчески сделает волосы густыми, блестящими и здоровыми, придавая им ухоженный вид и обеспечивая комфортное расчесывание каждый день.","Мы работаем над скинченджером (онлайновый - видят все те кто играет с мода)Его текущая версия доступна в VIP версии мода (170 рублей)Также в VIP, можно поменять возраст аккаунта, что даст вам возможность писать в чат, если вы не можете.Информация об покупке випа тут",0.40,средне релевантно
2,"Представляем новую версию генератора аватарок RoboFace, позволяющую создавать уникальные портреты персонажей всего за несколько кликов. Уникальные алгоритмы обеспечивают разнообразие лиц, причесок и выражений, позволяя легко подобрать идеальный образ для ваших игровых персонажей или соцсетей. С RoboFace теперь легко и быстро получить персонализированное изображение высокого качества.","Мы работаем над скинченджером (онлайновый - видят все те кто играет с мода)Его текущая версия доступна в VIP версии мода (170 рублей)Также в VIP, можно поменять возраст аккаунта, что даст вам возможность писать в чат, если вы не можете.Информация об покупке випа тут",0.95,очень релевантно
3,"Современная система автоматического полива растений позволяет забыть о ручном поливе и обеспечить вашим зеленым друзьям оптимальное количество влаги ежедневно. Оснащенная датчиками влажности почвы, она регулирует подачу воды в зависимости от потребностей растения, предотвращая пересыхание и переувлажнение грунта. Простота установки и интуитивно понятное управление делают систему идеальным решением для занятых садоводов и любителей комнатных цветов.","Доброе утроЕсли Инстаграмм удалят, мы сможем продолжить с вами общение здесьдавно хотела его завестинастало время, да и потом, здесь можно выкладывать гораздо больше и никто тебя за это не заблокирует",0.04,совершенно нерелевантно
4,"Стильный портативный вентилятор поможет создать комфортные условия во время отдыха дома или на даче. Эргономичный дизайн и компактные размеры позволяют легко брать устройство с собой куда угодно. Мощный поток воздуха обеспечивает свежесть даже в жаркие дни, делая использование вентилятора приятным и полезным дополнением к вашему дому.","Доброе утроЕсли Инстаграмм удалят, мы сможем продолжить с вами общение здесьдавно хотела его завестинастало время, да и потом, здесь можно выкладывать гораздо больше и никто тебя за это не заблокирует",0.63,средне релевантно
5,"Персональный фитнес-трекер помогает следить за физической активностью, сном и здоровьем сердца. С помощью простого интерфейса и удобных приложений трекер предоставляет детальную статистику ваших тренировок и сна, позволяя лучше понимать свое тело и улучшать результаты занятий спортом. Благодаря компактному размеру и водонепроницаемости, фитнес-трекер удобно носить каждый день, поддерживая активный образ жизни и заботу о себе.","Доброе утроЕсли Инстаграмм удалят, мы сможем продолжить с вами общение здесьдавно хотела его завестинастало время, да и потом, здесь можно выкладывать

In [ ]:

df.to_csv("data/golden/golden_pairs.csv", encoding="utf-8-sig", sep='\t')